# Configuring IP

This example notebook shows how a user can configure the IP address of their board with UDP or FTDI. 

**Refer to *Networking* in the *Advanced* section of the Naluscope manual for assistance with setting up an Ethernet connection to your board.**

**A custom register configuration file (*register-config.yml*) will be generated by the notebook and must be used for future connections to board for as long as the IP configured with this notebook is valid or until the next reboot of the board.**


## Naludaq Version

MinVersion: 0.33.2

In [ ]:
# Print Naludaq version
import naludaq

print(f"Naludaq version: {naludaq.__version__}")

## Compatible Boards

* This example applies to board models with USB or Ethernet ports.

## Imports and Variables

In [ ]:
from naluconfigs import get_available_models
from naludaq.board import Board
from naludaq.communication import ControlRegisters
from naludaq.controllers import get_connection_controller

## Logging

The logger setup below will show additional information from the NaluDAQ package. This is useful for debugging purposes, but
may be skipped if not needed.

In [ ]:
import logging


def setup_logger(level=logging.INFO):
    """Setup a basic logger."""
    logger = logging.getLogger()
    handler = logging.StreamHandler()
    handler.setFormatter(
        logging.Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s")
    )
    logger.addHandler(handler)
    logger.setLevel(level)
    suppress = [
        "naludaq.UART",
        "naludaq.FTDI",
    ]
    for name in suppress:
        logging.getLogger(name).setLevel(logging.CRITICAL)
    return logger


logger = setup_logger()

try:
    logger.debug("logger already setup")
except Exception:
    logger = setup_logger()

## Options

A list of valid board models can be found using the following code snippet:


In [ ]:
# Determining Board Model
print(get_available_models())


The following parameters must be provided to successfully set the IP address:

- **board_model**:       *board_model*
  - Valid options can be found using the command in the previous code block.

- **connection_method**:      *connection_method*
  - This should be "udp" or "ftdi". Other connections methods may be possible but are not supported or implemented in this notebook.

- **default_board_addr**: (*ip_address*, *port*)
  - Refer to the Quick Start guide for your board.

- **target_board_addr**:  (*ip_address*, *port*)
  - This is the IP address that you would like to set for the board

- **recv_addr**:          (*ip_address*, *port*)
   - This is the IP address of the computer used to configure the IP address of the board

Set these parameters in the configuration block below. All file and folder names are assumed to be in the same directory as the notebook.

**Note: The code in this notebook is only valid for changing from the board's default IP address to a different IP address and does not support changing to a different IP address from a previous manually configured IP address.**

In [ ]:
################
# Board Config #
################
board_model: str = "dsa-c10-8"
connection_method: str = "udp"

# Validate connection methods
connection_methods: list[str] = ["udp", "ftdi"]
if connection_method.lower() not in connection_methods:
    print(f"Invalid connection method. Valid connection methods: {connection_methods}")


#############
# IP Config #
#############
target_board_addr: tuple[str, int] = ("192.168.1.66", 4660)
recv_addr: tuple[str, int] = ("192.168.1.124", 4660)

##############
# UDP Config #
##############
default_board_addr: tuple[str, int] = ("192.168.1.64", 4660)

###############
# FTDI Config #
###############
serial_number: str = "A503VYQF"

Loads the default register and clock configuration.

In [ ]:
board = Board(board_model)

Converts *target_ip_addr* into hex codes for applicable registers.

In [ ]:
src_ip, src_port = target_board_addr
dest_ip, dest_port = recv_addr
src_octets = get_connection_controller(board)._get_octets_or_raise(src_ip)
dest_octets = get_connection_controller(board)._get_octets_or_raise(dest_ip)

eth_src_addr31_16 = (src_octets[0] << 8) | src_octets[1]
eth_src_addr15_0 = (src_octets[2] << 8) | src_octets[3]

eth_dest_addr31_16 = (dest_octets[0] << 8) | dest_octets[1]
eth_dest_addr15_0 = (dest_octets[2] << 8) | dest_octets[3]

eth_src_addr_sel = 1
eth_dest_addr_sel = 1
ethernet_src_config = {
    "eth_src_addr31_16": eth_src_addr31_16,
    "eth_src_addr15_0": eth_src_addr15_0,
    "eth_dest_addr31_16": eth_dest_addr31_16,
    "eth_dest_addr15_0": eth_dest_addr15_0,
    "eth_src_port": src_port,
    "eth_dest_port": dest_port,
    "eth_src_addr_sel": eth_src_addr_sel,
    "eth_dest_addr_sel": eth_dest_addr_sel,
    "tx_mode": 1,
}

In [ ]:
ethernet_src_config

Open connection to board. Connection method is specified in the Options section of this notebook.

In [ ]:
if connection_method == "udp":
    board.get_udp_connection(default_board_addr, recv_addr)
    board.disconnect()
elif connection_method == "ftdi":
    board.get_ftdi_connection(serial_number=serial_number, baud=115200)
    board.disconnect()
else:
    print(f"{connection_method} is not valid or is not implemented in this notebook.")

Starts the board and writes *target_ip_address* to board. 

In [ ]:
board.disconnect()

In [ ]:
with board:
    try:
        ControlRegisters(board).write_many(ethernet_src_config)
    except Exception as e:
        print(f"Failed due to {e}")

At this point the board has a new IP, you can either create a new board object, or reuse the old one with a new connection.

THe board at this point is disconnected and a new connection can be created.

In [ ]:
board.start_server(".")
board.connect_udp(
    board_addr=target_board_addr,
    receiver_addr=recv_addr,
)

print(board.initialize())
board.disconnect()

In [ ]:
with board:
    print(ControlRegisters(board).read("identifier"))